# Comparing a USGSCSM and ISIS camera for Dawn FC

In [ ]:
import pvl
import numpy as np
import os
import pandas as pd
import knoten
import csmapi
import pyproj

from ale import vis

from pysis import isis
from pysis.exceptions import ProcessError

## Make a CSM sensor model

In [ ]:
imageLink = 'https://pdsimage.wr.usgs.gov/Missions/Dawn/Ceres/DWNCSFC2_1A/DATA/IMG/20150604_SURVEY/20150617_CYCLE5/FC21A0038582_15170161546F6F.IMG'
!wget -N -P data/ {imageLink}
fileName = 'data/' + os.path.split(imageLink)[1]

camera = knoten.csm.create_csm(fileName)

## Ingest the image and spiceinit

In [ ]:
# Set the output location of the resulting .cub
cub_loc = os.path.splitext(fileName)[0] + '.cub'

try: 
    isis.dawnfc2isis(from_=fileName, to=cub_loc)
except ProcessError as e:
    print(e.stderr)

try:
    isis.spiceinit(from_=cub_loc, shape='ellipsoid')
except ProcessError as e:
    print(e.stderr)

## Define a function that compares ISIS and USGSCSM pixels

In [ ]:
def check_pixel_isis_ground(camera, cub, line, sample):
    """Compares ISIS and USGSCSM pixel.
    
    Takes an image coordinate, projects it to a ground point using ISIS, then projects
    the result back into an image coordinate using USGSCSM and computes the difference
    between image coordinates.
    """
    try:
        output = isis.campt(from_=cub, line=line, sample=sample, allowoutside='Yes')
    except ProcessError as e:
        print(e.stderr)
        return np.NaN, np.NaN
    pvl_output = pvl.loads(output)
    bodyfixed = pvl_output['GroundPoint']['BodyFixedCoordinate']
    bodyfixed = np.asarray(bodyfixed.value) * 1000
    image_coord = camera.groundToImage(csmapi.EcefCoord(*bodyfixed))
    # (.5,.5) in CSM == (1,1) in ISIS, so we have to subtract (.5,.5) from the ISIS pixels
    line_diff = line - image_coord.line - .5
    sample_diff = sample - image_coord.samp - .5
    return line_diff, sample_diff

def check_pixel_csm_ground(camera, cub, line, sample):
    """Compares USGSCSM pixel to ISIS.
    
    Takes an image coordinate, projects it to a ground point using CSM, then projects
    the result back into an image coordinate using ISIS and computes the difference
    between image coordinates.
    """
    image_coord = csmapi.ImageCoord(line, sample)
    ecef_coord = camera.imageToGround(image_coord, 0.0)
    
    def ecef_to_lla(camera, ecef_coord):
        semi_major, semi_minor = knoten.csm.get_radii(camera)
        ecef = pyproj.Proj(proj='geocent', a=semi_major, b=semi_minor)
        lla = pyproj.Proj(proj='latlong', a=semi_major, b=semi_minor)
        lon, lat, alt = pyproj.transform(ecef, lla, ecef_coord.x, ecef_coord.y, ecef_coord.z)
        return lon, lat, alt
    
    lon, lat, _ = ecef_to_lla(camera, ecef_coord)
    try:
        output = isis.campt(from_=cub, type='ground', latitude=lat, longitude=lon, allowoutside='Yes')
        pvl_output = pvl.loads(output)
        isis_line = pvl_output['GroundPoint']['Line']
        isis_sample = pvl_output['GroundPoint']['Sample']
    except ProcessError as e:
        print(e)
        isis_line = np.NaN
        isis_sample = np.NaN
    # (.5,.5) in CSM == (1,1) in ISIS, so we have to add (.5,.5) from the ISIS pixels
    line_diff = isis_line - image_coord.line + .5
    sample_diff = isis_sample - image_coord.samp + .5
    return line_diff, sample_diff

In [ ]:
def check_orientation(camera, cub, line, sample):
    try:
        output = isis.campt(from_=cub, line=line, sample=sample)
    except ProcessError as e:
        print(e.stderr)
        return [np.NaN, np.NaN, np.NaN], [np.NaN, np.NaN, np.NaN]
    pvl_output = pvl.loads(output)
    isis_position = np.array(pvl_output['GroundPoint']['SpacecraftPosition'].value) * 1000
    isis_look = np.array(pvl_output['GroundPoint']['LookDirectionBodyFixed'])
    csm_locus = camera.imageToRemoteImagingLocus(csmapi.ImageCoord(line - 0.5, sample - 0.5))
    csm_position = np.array([csm_locus.point.x, csm_locus.point.y, csm_locus.point.z])
    csm_look = np.array([csm_locus.direction.x, csm_locus.direction.y, csm_locus.direction.z])
    diff_position = isis_position - csm_position
    diff_look = isis_look - csm_look
    return diff_position, diff_look

## Get the total number of lines / samples

In [ ]:
isis_label = pvl.load(cub_loc)
n_samples = isis_label['IsisCube']['Core']['Dimensions']['Samples']
n_lines = isis_label['IsisCube']['Core']['Dimensions']['Lines']

## Compare top left, top right, bottom left, bottom right, and center pixels using check_pixel

In [ ]:
pixels_dict = {'line' : [0,0,n_lines, n_lines, n_lines/2],
               'sample' : [0, n_samples, 0, n_samples, n_samples/2]}

pixels_df = pd.DataFrame.from_dict(pixels_dict)
pixels_df['line_diff'] = np.NaN
pixels_df['sample_diff'] = np.NaN
pixels_df['pos_x_diff'] = np.NaN
pixels_df['pos_y_diff'] = np.NaN
pixels_df['pos_z_diff'] = np.NaN
pixels_df['look_x_diff'] = np.NaN
pixels_df['look_y_diff'] = np.NaN
pixels_df['look_z_diff'] = np.NaN

for idx, row in pixels_df.iterrows():
    pixels_df.iloc[idx]['line_diff'], pixels_df.iloc[idx]['sample_diff'] = check_pixel_isis_ground(camera, cub_loc, row['line'], row['sample'])
    diff_position, diff_look = check_orientation(camera, cub_loc, row['line'], row['sample'])
    pixels_df.iloc[idx]['pos_x_diff'] = diff_position[0]
    pixels_df.iloc[idx]['pos_y_diff'] = diff_position[1]
    pixels_df.iloc[idx]['pos_z_diff'] = diff_position[2]
    pixels_df.iloc[idx]['look_x_diff'] = diff_look[0]
    pixels_df.iloc[idx]['look_y_diff'] = diff_look[1]
    pixels_df.iloc[idx]['look_z_diff'] = diff_look[2]

pixels_df

## Generate a more detailed comparison
The level of detailed can be changed through the nx and ny variables in the reprojection_diff function

In [ ]:
csm_isd = os.path.splitext(fileName)[0] + '.json'

plot1, plot2, plot3, df1, df2, df3 = vis.reprojection_diff(csm_isd, cub_loc, nx=4, ny=8)
(plot1+plot2+plot3).cols(2)